In [1]:
import torch
from dataset import MovieDataset, PadCollate
from torch.utils.data import DataLoader
from model import MoviePredict
import pickle

In [2]:
movie_dataset = MovieDataset('../train_data.json')

In [3]:
dataloader = DataLoader(movie_dataset, collate_fn=PadCollate(), batch_size=3, shuffle=True)

In [5]:
with open('../meta_data.pkl', 'rb') as f:
    data = pickle.load(f)
    
voca_num = len(data['vocab'])
actor_num = len(data['actors'])
director_num = len(data['directors'])
category_num = len(data['categories'])
country_num = len(data['countries'])

In [6]:
model = MoviePredict(voca_num, actor_num, director_num, category_num, country_num)

In [7]:
input_ = next(iter(dataloader))[:-1]
story = input_[-1]

In [8]:
word_embs = model.word_embedding(story)

In [9]:
from torch import nn
conv = nn.Conv1d(1, 128, (4, 256))

In [10]:
conv(word_embs.unsqueeze(1)).shape

torch.Size([3, 128, 297, 1])

In [11]:
conv_norm = nn.BatchNorm1d(128)

In [12]:
conv_norm(conv(word_embs.unsqueeze(1)).squeeze(3))

tensor([[[-1.6841, -0.6061, -0.2072,  ..., -0.0089, -0.0089, -0.0089],
         [ 1.3599, -0.0505,  0.6139,  ...,  0.0311,  0.0311,  0.0311],
         [ 0.5915,  0.1102, -0.9692,  ..., -0.0791, -0.0791, -0.0791],
         ...,
         [-0.8811,  0.9990,  0.8670,  ..., -0.0847, -0.0847, -0.0847],
         [-1.5680, -0.7995, -2.0077,  ..., -0.0037, -0.0037, -0.0037],
         [ 0.8684,  0.0218,  0.7606,  ...,  0.0212,  0.0212,  0.0212]],

        [[ 0.8591, -1.0786, -0.7907,  ..., -0.1521,  0.2953, -1.0002],
         [ 0.8362,  0.1380,  0.0977,  ..., -0.5715,  0.9822,  0.3155],
         [ 0.5375, -0.0043, -1.5032,  ...,  1.6307,  0.1475,  1.9939],
         ...,
         [ 1.4155, -1.1499, -0.0315,  ..., -0.2692,  0.9232, -0.0163],
         [-1.3947, -2.0076, -0.5071,  ...,  0.9779, -0.5269, -0.0319],
         [ 0.6680, -0.3207,  1.6441,  ...,  0.1696, -0.1910, -0.1247]],

        [[-1.5656,  1.0935,  0.6706,  ..., -0.0089, -0.0089, -0.0089],
         [ 0.1580, -1.3383,  1.1796,  ...,  0

In [13]:
dropout = nn.Dropout(0.3)

In [14]:
nn.functional.adaptive_max_pool1d(dropout(conv_norm(conv(word_embs.unsqueeze(1)).squeeze(3))), 1).shape

torch.Size([3, 128, 1])

In [16]:
model(*input_).shape

torch.Size([3, 1])

In [17]:
gru = nn.GRU(256, 256, num_layers=4, batch_first=True, bidirectional=True)

In [19]:
output, h_n = gru(word_embs)

In [20]:
h_n.shape

torch.Size([8, 3, 256])